In [ ]:
import pandas as pd
import datetime
from waipawama import Finance
import pathlib
import datetime as dt
import redis

from fastcounting import helper

In [ ]:
r = redis.Redis(**helper.Helper().rediscred, decode_responses=True)

In [ ]:
lua_script = """
local result = 0.0
for l, atomic in ipairs(redis.call('SMEMBERS', KEYS[1]))
do result = result + redis.call('HGET', 'atomicID:' .. atomic, 'amount')
end
return result
"""

In [ ]:
result = r.eval(lua_script, 1, 'account:atomic:3300.0')

In [ ]:
result

In [ ]:
r.hgetall('generalID:10')

In [ ]:
r.hgetall('atomicID:2')

In [ ]:
def read_lexware(month, name='report', nrows=None):
    """Read xlxs from the default folder for each year e.g. month=2018-13 or actual month."""
    p = helper.Helper().datafolder(month)
    files = [file for file in p.iterdir() if file.parts[-1].lower().startswith(name)]
    if files:
        data = pd.read_table(files[0], sep='\t', engine='python', header=[0, 1], decimal=',', thousands='.', parse_dates=[2])
    return data

In [ ]:
def clean_report(validate):
    better_columns = ['Konto', 'Name', 'Letzte Buchung', 'EB Soll', 'EB Haben', 'Summe Soll', 'Summe Haben', 'drop1', 'drop2', 'Saldo Soll', 'Saldo Haben']
    validate.columns = better_columns
    validate.set_index('Konto', inplace = True)
    validate.fillna(0, inplace=True)  # for full year data set they are equal to sum
    validate.drop(['drop1', 'drop2'], axis=1, inplace=True)
    return validate

In [ ]:
# main
validate = read_lexware('2017-13')
validate = clean_report(validate)

In [ ]:
# check total sum which is aggregation of EB + Saldo Haben + Saldo Soll
# For open 

In [ ]:
validate['checksum'] =  validate['Saldo Haben'] - validate['Saldo Soll']

In [ ]:
redis_sum_list = []
for konto in validate.index:
    redis_sum_list.append(r.eval(lua_script, 1, f'account:atomic:{float(konto)}'))

In [ ]:
validate['test_redis'] = redis_sum_list
validate['test_redis'] = validate['test_redis'] / 100

In [ ]:
validate

In [ ]:
validate[validate['checksum'] != validate['test_redis']]

In [ ]:
r.smembers('account:atomic:10007.0')

In [ ]:
r.hgetall('atomicID:139')